### 練習問題2.43
Louis Reasonerは練習問題2.42を解くのにだいぶ苦労している。  
彼が書いたqueens⼿続きは動いてはいるようだが、  
⾮常に遅いのだ(6×6の場合でも、Louisは待ちきれなくなっ てしまう)。  
LouisがEva Lu Atorに⾒てくれるよう頼むと、  
彼⼥はLouisがflatmap内のマップのネストの順番を次のように逆にしてしまっていることを指摘した。 

    (flatmap (lambda (new-row)
                (map (lambda (rest-of-queens)
                            (adjoin-position new-row k rest-of-queens))
                     (queen-cols (- k 1)))
                     )
             (enumerate-interval 1 board-size)
             )  

逆にするとなぜプログラムの実⾏が遅くなるのか説明せよ。  
練習問題2.42のプログラムが8クイーンパズルを解く時間をTとして、   
Louisのプログラムがパズルを解くのにかかる時間を⾒積もれ。

In [1]:
; 問題文のqueens手続き
; 以下の定義が必要
; ・empty-board
; ・safe?手続き
; ・adjoin-position手続き
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
          (flatmap (lambda (rest-of-queens)
                    (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
                         (enumerate-interval 1 board-size))
                    )
                  (queen-cols (- k 1))
          )
         )
        )
    )
  (queen-cols board-size)
  )

(define (flatmap proc seq)
  (accumulate append '() (map proc seq)))

; 整数列の列挙
(define (enumerate-interval low high)
  (if (> low high) '()
      (cons low (enumerate-interval (+ low 1) high))))

; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence)))))

; フィルタリング
(define (filter predicate sequence)
  (cond ((null? sequence) '())
        ((predicate (car sequence))(cons (car sequence) (filter predicate (cdr sequence))))
        (else (filter predicate (cdr sequence))))
  )

; 回答1
(define empty-board ())

; コンストラクタとセレクタ
(define (make-pos x y)(cons x y))
(define (x-pos p)(car p))
(define (y-pos p)(cdr p))

; 回答2
; adjoin-position手続きの呼び出し方が決まっているため、
; この回答しかないはず。
(define (adjoin-position new-row k rest-of-queens)
  (append rest-of-queens (list (make-pos k new-row)))
  )
  
(define k 4)
(define board-size 4)
(define rest-of-queens (list (make-pos 1 2) (make-pos 2 4) (make-pos 3 1)))
(map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
     (enumerate-interval 1 board-size))


; k列目の位置のy座標を返す
(define (get-k-y k positions)
  (define (iter count pos)
    (if (>= count k) (y-pos (car pos))
        (iter (+ count 1) (cdr pos))
        )
    )
  (iter 1 positions)
  )

; 回答3
(define (safe? k positions)
  (begin
   (display positions)
   (newline)
   (let ((k-y-pos (get-k-y k positions)))
    (define (iter count pos)
      (if (>= count k) #t
        (if (null? pos) #t
            (let ((y (y-pos (car pos))))
              (let ((y-upper (- y (- k count)))
                    (y-lower (+ y (- k count))))
                ; y座標が同じか、対角線上にあればFalseを返す。
                ; それ以外はチェックを継続する。
                (if (or (= k-y-pos y) (= k-y-pos y-upper) (= k-y-pos y-lower)) #f
                    (iter (+ count 1) (cdr pos))
                    )
                )
              )
            )
        )
      )
      (iter 1 positions)
    )
   )
  )


In [2]:
(define (queens2 board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
          ;(flatmap (lambda (rest-of-queens)
          ;          (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
          ;               (enumerate-interval 1 board-size))
          ;          )
          ;        (queen-cols (- k 1))
          ;)
         (flatmap (lambda (new-row)
                    (map (lambda (rest-of-queens)
                           (adjoin-position new-row k rest-of-queens))
                         (queen-cols (- k 1)))
                    )
                  (enumerate-interval 1 board-size)
                  )  
         )
        )
    )
  (queen-cols board-size)
  )


In [3]:
(define (loop max proc n)
  (define (loop-iter count)
    (if (< count max)(begin
                  ;(display (proc n))
                  ;(newline)
                  (proc n)
                  (loop-iter (+ count 1))
        )
    )
   )
  (loop-iter 0)
  )

In [4]:
%%time

(loop 1 queens 4)

((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1) (2 . 1))
((1 . 1) (2 . 2))
((1 . 1) (2 . 3))
((1 . 1) (2 . 4))
((1 . 2) (2 . 1))
((1 . 2) (2 . 2))
((1 . 2) (2 . 3))
((1 . 2) (2 . 4))
((1 . 3) (2 . 1))
((1 . 3) (2 . 2))
((1 . 3) (2 . 3))
((1 . 3) (2 . 4))
((1 . 4) (2 . 1))
((1 . 4) (2 . 2))
((1 . 4) (2 . 3))
((1 . 4) (2 . 4))
((1 . 1) (2 . 3) (3 . 1))
((1 . 1) (2 . 3) (3 . 2))
((1 . 1) (2 . 3) (3 . 3))
((1 . 1) (2 . 3) (3 . 4))
((1 . 1) (2 . 4) (3 . 1))
((1 . 1) (2 . 4) (3 . 2))
((1 . 1) (2 . 4) (3 . 3))
((1 . 1) (2 . 4) (3 . 4))
((1 . 2) (2 . 4) (3 . 1))
((1 . 2) (2 . 4) (3 . 2))
((1 . 2) (2 . 4) (3 . 3))
((1 . 2) (2 . 4) (3 . 4))
((1 . 3) (2 . 1) (3 . 1))
((1 . 3) (2 . 1) (3 . 2))
((1 . 3) (2 . 1) (3 . 3))
((1 . 3) (2 . 1) (3 . 4))
((1 . 4) (2 . 1) (3 . 1))
((1 . 4) (2 . 1) (3 . 2))
((1 . 4) (2 . 1) (3 . 3))
((1 . 4) (2 . 1) (3 . 4))
((1 . 4) (2 . 2) (3 . 1))
((1 . 4) (2 . 2) (3 . 2))
((1 . 4) (2 . 2) (3 . 3))
((1 . 4) (2 . 2) (3 . 4))
((1 . 1) (2 . 4) (3 . 2) (4 . 1))
((1 . 1) (2 . 

#f

In [5]:
%%time

(loop 1 queens2 4)

((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1) (2 . 1))
((1 . 2) (2 . 1))
((1 . 3) (2 . 1))
((1 . 4) (2 . 1))
((1 . 1) (2 . 2))
((1 . 2) (2 . 2))
((1 . 3) (2 . 2))
((1 . 4) (2 . 2))
((1 . 1) (2 . 3))
((1 . 2) (2 . 3))
((1 . 3) (2 . 3))
((1 . 4) (2 . 3))
((1 . 1) (2 . 4))
((1 . 2) (2 . 4))
((1 . 3) (2 . 4))
((1 . 4) (2 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1) (2 . 1))
((1 . 2) (2 . 1))
((1 . 3) (2 . 1))
((1 . 4) (2 . 1))
((1 . 1) (2 . 2))
((1 . 2) (2 . 2))
((1 . 3) (2 . 2))
((1 . 4) (2 . 2))
((1 . 1) (2 . 3))
((1 . 2) (2 . 3))
((1 . 3) (2 . 3))
((1 . 4) (2 . 3))
((1 . 1) (2 . 4))
((1 . 2) (2 . 4))
((1 . 3) (2 . 4))
((1 . 4) (2 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 . 3))
((1 . 4))
((1 . 1))
((1 . 2))
((1 

#f